In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [103]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os
from numpy.linalg import inv
import svdt

%matplotlib inline

In [104]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [105]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [106]:
# machinery to find rotation and translation with SVD:

# Matrix of empirically found positions        F: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
# Rotation matrix                              R
# Matrix of estimated, translated positions    T: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]

# fake data
T_estimate = np.array([[0.5, 0.5*np.sqrt(3)/2., 0.], [-0.5*np.sqrt(3)/2., 0.5, 0.], [-0.5, -0.5*np.sqrt(3)/2.,0.]])
F = np.array([[0, 1., 0],[-1, 0, 0],[0, -1, 0]])


R, L, RMSE = svdt.svd(A=T_estimate, B=F)

print(R)
print(L)
print(RMSE)

[[ 0.65465367 -0.75592895  0.        ]
 [ 0.75592895  0.65465367  0.        ]
 [ 0.          0.          1.        ]]
[-1.12854057e-01 -2.77555756e-17  0.00000000e+00]
0.18428990404026965


In [75]:
L

array([ 0.70601613, -0.94893617,  0.        ])

In [72]:
coord_mapping_brightest.to_array()

AttributeError: 'DataFrame' object has no attribute 'to_array'

In [ ]:
F_matrix = np.array(())

In [8]:
def coords_brightest(star_positions_scrambled_ids_pass, num_rank):
    '''
    Return the coordinates of the brightest stars in the dataframe (column "Flux")
    
    INPUTS:
    star_positions_scrambled_ids_pass: DataFrame with xcentroid, ycentroid, and flux info
    num_rank: Nth brightest stars returned (i.e., 3 -> coordinates of 3 brightest are returned)
    
    OUTPUTS:
    2D array of x,y coordinates of the Nth brightest stars
    '''
    
    brightest_fluxes = np.sort(star_positions_scrambled_ids_pass["flux"])[-int(num_rank):]
    #print("brightest")
    #print(brightest_fluxes)
    # pick table rows where stars are at least as bright as the third-brightest star
    idx_cond = star_positions_scrambled_ids_pass["flux"]>=brightest_fluxes[0]
    # get their positions
    x_brights = star_positions_scrambled_ids_pass["xcentroid"][idx_cond]
    y_brights = star_positions_scrambled_ids_pass["ycentroid"][idx_cond]
    #print(star_positions_scrambled_ids_pass[idx_cond])
    coords_empir_brights = zip(x_brights,y_brights)
    
    return list(coords_empir_brights)

In [69]:
def match_stars(predicted_coords_pass, empirical_coords_pass):
    '''
    Matches up predicted and empirical coordinates of bright stars
    
    INPUTS:
    predicted_coords_pass: list of predicted positions of the brightest stars in the initial frame
    empirical_coords_pass: list of empirical coordinates of LOTS of stars in the current frame
    
    OUTPUTS:
    df_map: DataFrame with mapping of predicted and true coordinates
    '''
    
    # convert inputs to dataframes
    df_pred = pd.DataFrame(predicted_coords_pass, columns=["x_pred", "y_pred"])
    df_empir = pd.DataFrame(empirical_coords_pass, columns=["x_empir", "y_empir"])
    
    # initialize columns of 'true' coordinates to predicted dataframe
    df_pred["x_true"] = np.nan
    df_pred["y_true"] = np.nan

    
    # pluck out the 'true' coordinates from the empirical data
    for num_bright in range(0,len(df_pred)):
        # condition is "where difference in coordinates is less than N=3 pixels"
        idx_cond = np.sqrt(
                        np.add(
                            np.power(np.subtract(df_pred["x_pred"][num_bright],df_empir["x_empir"]),2.),
                            np.power(np.subtract(df_pred["y_pred"][num_bright],df_empir["y_empir"]),2.)
                            )
                        ) < 3
        
        # populate the 'true' columns of the DataFrame of predicted values
        df_pred["x_true"][num_bright] = df_empir["x_empir"].loc[idx_cond].values[0]
        df_pred["y_true"][num_bright] = df_empir["y_empir"].loc[idx_cond].values[0]
        
    df_map = df_pred.copy(deep=True)
        
    return df_map

In [9]:
# read in image displacements for ALL stars in ALL the frames
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (but with scrambled IDs)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)
#print(initial_star_pos)

# find the coordinates of the brightest three stars in the initial image;
# we will use this to figure out the net translation and rotation of successive frames
initial_coords_empir_brights = coords_brightest(initial_star_pos,num_rank=3) 
#print(initial_coords_empir_brights)

In [101]:
for num_star_file in range(0,len(file_names_stars)):
    # for each frame, calculate needed shifts of the aperture grid and take photometry

    # make copy of DataFrame of initial stars (we will change the x,y coords to make a prediction)
    predicted_star_pos = initial_star_pos.copy(deep=True)

    # read in current star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # read in empirically-found positions and photometry of ALL stars in this frame (but with scrambled IDs)
    star_info_scrambled_ids = pd.read_csv(file_names_pos[num_star_file], delim_whitespace=True)

    # retrieve (x,y) linear displacement vectors found independently for ALL stars in that frame, as was 
    # found by cross-correlating images
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift aperture grid of first (i.e., [0]) array of images by the linear displacement; this is a first-order 
    # correction to fit the current frame (n.b. we use this grid and translate it in order to preserve the ID 
    # information of the stars; otherwise, if we re-find the stars in each frame we don't know which is which)
    predicted_star_pos["xcentroid"] = initial_star_pos["xcentroid"]-x_disp
    predicted_star_pos["ycentroid"] = initial_star_pos["ycentroid"]-y_disp
    predicted_coords = list(zip(predicted_star_pos["xcentroid"],predicted_star_pos["ycentroid"]))

    # find the predicted coordinates (in this image) of the brightest three stars (as measured in the initial image)
    predicted_coords_brights = coords_brightest(predicted_star_pos,num_rank=3)

    # find the empirical coordinates of the brightest *twenty* stars in this image
    # (here we choose twenty, and from those we'll match the correct three, because the brightest
    # three stars in the first image are not necessarily the brightest three in every image)
    empirical_coords_brights = coords_brightest(star_info_scrambled_ids,num_rank=30)

    # match the stars by finding the sets of predicted and empirical coordinates that are within 3 (or so) pixels
    coord_mapping_brightest = match_stars(predicted_coords_brights, empirical_coords_brights)
    
    # use SVD decomposition to find the translation and rotation to minimize the residuals
    # Note syntax here:
        # Matrix of estimated, translated positions    A: [ [x_1 y_1 z_1=0], [x_2 y_2 z_2=0], [x_3 y_3 z_3=0] ]
        # Matrix of empirically found positions        B: [ [x_1' y_1' z_1'=0], [x_2' y_2' z_2'=0], [x_3' y_3' z_3'=0] ]
        # Rotation matrix                              R
        # Translation matrix                           T
    predicted_array = np.array([[coord_mapping_brightest["x_pred"][0], coord_mapping_brightest["y_pred"][0], 0.], 
                           [coord_mapping_brightest["x_pred"][1], coord_mapping_brightest["y_pred"][1], 0.], 
                           [coord_mapping_brightest["x_pred"][2], coord_mapping_brightest["y_pred"][2], 0.]])
    true_array = np.array([[coord_mapping_brightest["x_true"][0], coord_mapping_brightest["y_true"][0], 0.], 
                           [coord_mapping_brightest["x_true"][1], coord_mapping_brightest["y_true"][1], 0.], 
                           [coord_mapping_brightest["x_true"][2], coord_mapping_brightest["y_true"][2], 0.]])
    R, T, RMSE = svdt.svd(A=predicted_array, B=true_array)
    


    ## FYI
    '''
    apertures = CircularAperture(predicted_coords, r=4.)
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    plt.show()
    '''

    '''
    print("initial star pos")
    print(initial_star_pos)
    vectors_translation = np.transpose((np.subtract(initial_star_pos['xcentroid'],x_disp), 
                              np.subtract(initial_star_pos['ycentroid'],y_disp)))

    #print(vectors_translation.shape)
    print("vectors translation")
    print(vectors_translation)
    '''
    '''

    # get predicted (simple translation) coords of the brightest stars in the first frame
    # get empirical coords of the brightest stars in this frame
    df_vectors_translation = pd.DataFrame(vectors_translation, columns=["x_transl", "y_transl"])
    coords_predic_brights = coords_brightest(df_positions_translation)



    # take the three brightest ones

    # get empirical coords of the brightest stars in this frame
    #print(type(star_positions_scrambled_ids))
    coords_empir_brights = coords_brightest(star_positions_scrambled_ids)
    '''
    # get residuals from mapping of the predicted with the empirical coordinates
    # (then we know which star is which)
    #print(initial_coords_empir_brights)
    #print(coords_empir_brights)
    #print(coords_predic_brights)




    #print(star_positions_scrambled_ids.where[star_positions_scrambled_ids["xcentroid"]==brightest_fluxes[0]])["xcentroid"]

    '''


    ## ## fcn to find the positions of the three brightest stars closest to their estimated positions, 
    ## ## and with a brightness that is consistent (brightest stars have IDs 26, 272, 99 [I believe; check])

    file_names_pos
    '''
    ## ## insert SVD functionality here to find residual translation and rotation

    ## ## make the residual transformation to the aperture locations




    '''
    print(positions)

    apertures = CircularAperture(positions, r=4.)


    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output


    ## ## START TEST
    # convert to DataFrame
    df_phot = phot_table.to_pandas()

    # remove rows with coordinates that put the stars outside the 512x512 frame, or close to edge
    edge_buffer = 15 # pix
    #phot_table = phot_table[~mask]
    idx_edge_xs = np.logical_or(df_phot["xcenter"] > np.subtract(512,edge_buffer), df_phot["xcenter"] < edge_buffer)
    idx_edge_ys = np.logical_or(df_phot["ycenter"] > np.subtract(512,edge_buffer), df_phot["ycenter"] < edge_buffer)
    df_phot_drop_edges = df_phot.drop(df[np.logical_or(idx_edge_xs,idx_edge_ys)].index, inplace = False)

    # sort: brightest stars first
    df_phot_drop_edges = df_phot_drop_edges.sort_values("aperture_sum", axis=0, ascending=False)

    print(df_phot)
    print(df_phot_drop_edges)

    ## ## END TEST
    '''
    # write text file with aperture photometry
    '''
    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=False)
    print("Wrote out "+text_file_name)

    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''


    ## ## END FOR-LOOP

num star file
0
rotation:
[[1.00000000e+00 8.32667268e-17 0.00000000e+00]
 [1.11022302e-16 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
1.0000000000000004
translation:
[-1.70530257e-13 -1.13686838e-13  0.00000000e+00]
---
num star file
1
rotation:
[[ 1.00000000e+00 -1.38696574e-05  0.00000000e+00]
 [ 1.38696574e-05  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999999038166
translation:
[0.05670705 0.00695429 0.        ]
---
num star file
2
rotation:
[[ 9.99999998e-01 -5.64683971e-05  0.00000000e+00]
 [ 5.64683971e-05  9.99999998e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999984056598
translation:
[ 0.04853058 -0.03911157  0.        ]
---
num star file
3
rotation:
[[ 1.00000000e+00  1.66997789e-05  0.00000000e+00]
 [-1.66997789e-05  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999998605587
translation:
[0.0321869  0.02483955 0. 

rotation:
[[ 9.99999993e-01 -1.22362230e-04  0.00000000e+00]
 [ 1.22362230e-04  9.99999993e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999925137422
translation:
[ 0.03528144 -0.10270782  0.        ]
---
num star file
42
rotation:
[[ 9.99999885e-01 -4.79479527e-04  0.00000000e+00]
 [ 4.79479527e-04  9.99999885e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999998850496848
translation:
[ 0.13117504 -0.24379528  0.        ]
---
num star file
43
rotation:
[[ 9.99999968e-01 -2.52131170e-04  0.00000000e+00]
 [ 2.52131170e-04  9.99999968e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999682149362
translation:
[ 0.08941189 -0.1697918   0.        ]
---
num star file
44
rotation:
[[ 9.99999994e-01  1.05085985e-04  0.00000000e+00]
 [-1.05085985e-04  9.99999994e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999944784679
translation:
[-0.03064513 -0.01609751  0.        ]
-

rotation:
[[ 9.99999941e-01 -3.42818961e-04  0.00000000e+00]
 [ 3.42818961e-04  9.99999941e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999412375781
translation:
[ 0.09091091 -0.16393964  0.        ]
---
num star file
80
rotation:
[[ 9.99999976e-01 -2.17705070e-04  0.00000000e+00]
 [ 2.17705070e-04  9.99999976e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999999763022509
translation:
[ 0.06578472 -0.08316155  0.        ]
---
num star file
81
rotation:
[[ 9.99999698e-01 -7.77186946e-04  0.00000000e+00]
 [ 7.77186946e-04  9.99999698e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999996979901797
translation:
[ 0.18545339 -0.33056752  0.        ]
---
num star file
82
rotation:
[[ 9.99999784e-01 -6.56729908e-04  0.00000000e+00]
 [ 6.56729908e-04  9.99999784e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9999997843528908
translation:
[ 0.1580263  -0.31226538  0.        ]
-

IndexError: index 100 is out of bounds for axis 0 with size 100